In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 데이터 경로 변수

In [8]:
DATA_PATH = "/content/drive/MyDrive/final/data/"
DATA_PATH

'/content/drive/MyDrive/final/data/'

- 시드값

In [9]:
SEED = 42

- 데이터 불러오기

In [14]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [15]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 2310), (12225, 2310))

# 결측치 처리

In [16]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

,0


In [17]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

,0


In [ ]:
# train_ft['충동구매비율'] = train_ft['충동구매비율'].fillna(0)
# test_ft['충동구매비율'] = test_ft['충동구매비율'].fillna(0)

# train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [18]:
train_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,전체_성수기_구매비율,전체_준성수기_구매비율,전체_중간기_구매비율,전체_비수기_구매비율,성수기_구매집중도,계절_구매변동성,구매주기_대비_구매금액,방문당_평균구매금액,구매금액_대비_환불금액,환불건수_대비_구매건수
0,train_0,9,2004-05-07,2005-01-16,255,28,2,1.555556,22.744789,104,...,0.285714,0.142857,0.285714,0.214286,0.235294,0.123718,1.500429e+05,466800.000000,-0.599115,0.214286
1,train_1,21,2004-05-11,2005-04-27,352,16,1,2.000000,11.739134,3,...,0.285714,0.166667,0.214286,0.166667,0.244898,0.125236,3.151891e+05,240144.047619,0.000000,0.000000
2,train_2,55,2004-05-06,2005-04-26,356,6,11,1.854545,5.019987,4,...,0.137255,0.117647,0.264706,0.411765,0.097222,0.142635,3.747564e+06,408825.200000,-0.047698,0.068627
3,train_3,90,2004-05-11,2005-04-20,345,3,19,2.055556,3.137146,10,...,0.145946,0.200000,0.259459,0.286486,0.113445,0.089950,1.194659e+07,398219.577778,-0.080017,0.075676
4,train_4,24,2004-05-09,2005-03-17,313,13,6,1.500000,11.962731,44,...,0.138889,0.388889,0.194444,0.194444,0.116279,0.174212,3.068358e+05,166202.750000,-1.743453,0.361111


In [19]:
test_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,전체_성수기_구매비율,전체_준성수기_구매비율,전체_중간기_구매비율,전체_비수기_구매비율,성수기_구매집중도,계절_구매변동성,구매주기_대비_구매금액,방문당_평균구매금액,구매금액_대비_환불금액,환불건수_대비_구매건수
0,test_0,5,2004-05-16,2004-10-26,164,32,3,1.400000,30.416096,186,...,0.000000,0.142857,0.000000,0.571429,0.000000,0.179758,13424.343750,85915.800000,0.000000,0.000000
1,test_1,3,2004-09-11,2005-02-23,166,55,1,1.333333,57.879185,66,...,0.000000,0.000000,0.250000,0.500000,0.000000,0.353553,5283.636364,96866.666667,0.000000,0.000000
2,test_2,2,2004-08-16,2004-09-02,18,9,0,2.500000,7.361216,240,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.300000,67444.444444,303500.000000,0.000000,0.000000
3,test_3,14,2004-07-22,2005-04-19,272,19,1,1.714286,17.715031,11,...,0.041667,0.416667,0.208333,0.291667,0.032258,0.195434,121354.736842,164695.714286,0.473601,0.166667
4,test_4,7,2004-05-25,2005-02-13,265,37,3,2.142857,36.517818,76,...,0.466667,0.266667,0.000000,0.266667,0.368421,0.083887,42397.027027,224098.571429,0.237268,0.200000


# 특성 공학(Feature Engineering)

- ID 변수 제외

In [20]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 2309), (12225, 2309))

- 추가 피처 만들어 보기

In [ ]:
# cols = [ col for col in train_ft.columns if col.startswith("pivot_cnt_") ]
# cols

[]

In [ ]:
# train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
# train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
# train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

# test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
# test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
# test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2277), (12225, 2277))

In [ ]:
# train_ft.drop(columns=['중분류별_구매횟수_std', '중분류별_구매횟수_skew', '중분류별_구매횟수_kurt'], inplace=True)
# test_ft.drop(columns=['중분류별_구매횟수_std', '중분류별_구매횟수_skew', '중분류별_구매횟수_kurt'], inplace=True)

# train_ft.shape, test_ft.shape

((14940, 2274), (12225, 2274))

## Feature Encoding

In [21]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
첫구매날짜,337
마지막구매날짜,334
주구매지점,4
주구매_중분류,232
주구매_대분류,28
고객등급,5


#### 범주형피처 인코딩
- 첫/마지막구매 : ?
- 주구매 중/대분류 : category_encoding - count_encoding / onehot_encoding
- 주구매지점 : onehot_encoding / target_encoding
- 고객등급 : onehot_encoding / target_encoding / ordinal_encoding

In [22]:
# %pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
# import category_encoders as ce

In [ ]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점","주구매_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점","주구매_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 112), (12225, 112))

In [ ]:
enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft.shape, test_ft.shape

((14940, 113), (12225, 113))

- 문자열 피처 삭제

In [ ]:
cols

['주구매지점', '주구매_중분류', '주구매_대분류']

In [ ]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 110), (12225, 110))

In [ ]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

#### 스케일링

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
model = LogisticRegression(random_state=SEED)

scaler_cls_list = [StandardScaler, MinMaxScaler, PowerTransformer, RobustScaler]

# 스케일링 클래스별 점수 확인
def check_scaling_score(scaler_cls_list, x, y, model, scoring, cv):
  for scaler_cls in scaler_cls_list:
    scaler = scaler_cls()
    x_train = scaler.fit_transform(x)
    scores = cross_val_score(model, x_train, y, scoring=scoring, cv=cv, n_jobs=-1)
    print(scaler_cls.__name__ ,scores.mean())

In [ ]:
check_scaling_score(scaler_cls_list, train_ft, train_target, model, "f1_macro", cv)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

In [ ]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,월_방문비율,화_방문비율,수_방문비율,목_방문비율,금_방문비율,주말방문비율,18시이전방문비율,1월_방문비율,...,주구매_대분류_20,주구매_대분류_21,주구매_대분류_22,주구매_대분류_23,주구매_대분류_24,주구매_대분류_25,주구매_대분류_26,주구매_대분류_27,주구매_대분류_28,주구매_중분류_cnt
0,-0.437126,0.002987,-0.497583,-0.198628,0.634752,0.539937,-0.715699,0.257728,-0.071084,0.0,...,-0.081258,-0.18062,-0.171941,-0.124766,-0.074744,-0.08084,-0.089606,-0.101046,-0.048458,-0.726970
1,0.239394,-0.356452,-0.513898,0.908774,0.410631,0.713909,-0.384013,-1.008554,-0.332492,0.0,...,-0.081258,-0.18062,-0.171941,-0.124766,-0.074744,-0.08084,-0.089606,-0.101046,-0.048458,-0.510078
2,1.890101,-0.869935,0.903002,0.497355,-0.108385,-0.339076,-0.618782,0.036742,0.371936,0.0,...,-0.081258,-0.18062,-0.171941,-0.124766,-0.074744,-0.08084,-0.089606,-0.101046,-0.048458,-0.848408
3,4.298510,-1.023980,-0.125721,-0.169453,0.077106,0.136753,0.091374,-0.080558,0.048595,0.0,...,-0.081258,-0.18062,-0.171941,-0.124766,-0.074744,-0.08084,-0.089606,-0.101046,-0.048458,1.523851
4,0.618244,-0.613193,-0.177056,-0.110372,-0.326798,-0.773266,0.903624,0.302875,0.318500,0.0,...,-0.081258,-0.18062,-0.171941,-0.124766,-0.074744,-0.08084,-0.089606,-0.101046,-0.048458,-0.833157


# 정답 데이터

In [ ]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# cv 점수 확인해보기

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=5,shuffle=True, random_state=SEED)

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=SEED)
scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
np.mean(scores)

0.6837033656368326

# 모델 학습

In [ ]:
model = LGBMClassifier(random_state=SEED)
model.fit(train_ft,target)

[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007702 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 18138
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 109
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


LGBMClassifier(random_state=42)

# 테스트 데이터 예측

In [ ]:
pred = model.predict(test_ft)
pred

array([0., 0., 0., ..., 0., 0., 0.])

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [ ]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [ ]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,0.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


- 예측 결과를 csv 파일로 저장하여 제출

In [ ]:
submit.to_csv(f"{DATA_PATH}featurestest3.csv",index=False)